<a href="https://colab.research.google.com/github/sezinbiner/ClassificationOfCovidXray/blob/main/BitirmeV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cv2
import os
DATADIR = "/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset" 
classes = ["COVID","Normal"] 

X = []
y = []

for class_name in classes: 
  path = os.path.join(DATADIR,class_name)
  print(path)
  os.chdir(path) 
  for im in os.listdir(path):
      try:
          img = cv2.imread(im,cv2.IMREAD_COLOR)     
          X.append(img)      
      except Exception as e:
          pass

/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset/COVID


In [ ]:
print("selin")

In [ ]:
len(X)

13808